In [1]:
image_data = []
labels = []

label_dict = {
    "Aerodactyl": 0,
    "Bulbasaur": 1,
    "Charmander": 2,
    "Dratini": 3,
    "Fearow": 4,
    "Meowth": 5,
    "Pikachu": 6,
    "Psyduck": 7,
    "Spearow": 8,
    "Squirtle": 9,
}

In [2]:
import os
folders = os.listdir("Train")
print(folders)

['Aerodactyl', 'Bulbasaur', 'Charmander', 'Dratini', 'Fearow', 'Meowth', 'Pikachu', 'Psyduck', 'Spearow', 'Squirtle']


In [3]:
from keras.preprocessing import image

In [4]:
for ix in folders:
  path = os.path.join("Train", ix)
  for im in os.listdir(path):
    img = image.load_img(os.path.join(path,im), target_size=(224, 224))
    img_array = image.img_to_array(img)
    image_data.append(img_array)
    labels.append(label_dict[ix])

c:\users\linta.kumar\appdata\local\programs\python\python37\lib\site-packages\PIL\Image.py:952: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [5]:
print(len(image_data))

1645


In [6]:
print(len(labels))

1645


In [7]:
import random
combined = list(zip(image_data, labels))
random.shuffle(combined)

In [8]:
image_data[:], labels[:] = zip(*combined)

In [9]:
print(len(image_data))

1645


In [10]:
import numpy as np

In [11]:
X_train = np.array(image_data)
Y_train = np.array(labels)

print(X_train.shape, Y_train.shape)

(1645, 224, 224, 3) (1645,)


In [12]:
from keras.utils import to_categorical

In [13]:
Y_train = to_categorical(Y_train)

In [14]:
Y_train[:10]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [15]:
X = X_train.reshape((-1,224,224,1))

In [16]:
print(X.shape, Y_train.shape)

(4935, 224, 224, 1) (1645, 10)


In [17]:
X_train = X_train/255

In [18]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import np_utils
import tensorflow
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam

In [19]:
model = Sequential()
model.add(Convolution2D(32,(3,3), activation='relu',input_shape=(224,224,3)))
model.add(Convolution2D(64,(3,3), activation='relu'))
model.add(Dropout(.25))
model.add(MaxPooling2D(2,2))
model.add(Convolution2D(32,(5,5), activation='relu'))
model.add(Convolution2D(8,(5,5), activation='relu'))
model.add(Flatten())
model.add(Dense(10,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
dropout (Dropout)            (None, 220, 220, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 106, 106, 32)      51232     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 102, 102, 8)       6408      
_________________________________________________________________
flatten (Flatten)            (None, 83232)             0

In [20]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
vggmodel = VGG16(weights='imagenet', include_top= False, input_shape=(224,224, 3))

In [22]:
for layer in vggmodel.layers:
    layer.trainable = False

In [23]:
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [24]:
av1 = GlobalAveragePooling2D()(vggmodel.output)
fc1 = Dense(256, activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(10, activation='softmax')(d1)

In [25]:
model_new = Model(inputs = vggmodel.input , outputs=fc2)
model_new.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [26]:
adam = Adam(lr=0.00003)
model_new.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [27]:
feature_extractor = vggmodel.predict(X_train)

In [28]:
features = feature_extractor.reshape(feature_extractor.shape[0],-1)

In [29]:
X_for_RF = features

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
rfmodel = RandomForestClassifier(n_estimators = 50, random_state = 2)

In [33]:
rfmodel.fit(X_for_RF, Y_train)

RandomForestClassifier(n_estimators=50, random_state=2)

In [34]:
rfmodel.score(X_for_RF, Y_train)

0.990273556231003

In [35]:
testfeature_extractor = vggmodel.predict(X_test)
testfeatures = testfeature_extractor.reshape(testfeature_extractor.shape[0],-1)

NameError: name 'X_test' is not defined

In [36]:
predictions = rfmodel.predict(testfeatures)

NameError: name 'testfeatures' is not defined

In [569]:
predictions

array([0, 0, 2, 6, 6, 6, 6, 6, 6, 6, 6, 1, 6, 1, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 7, 7, 7, 6, 6, 5, 6, 7,
       4, 7, 1, 7, 2, 7, 7, 1, 2, 8, 8, 8, 8, 1, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 9, 1, 9, 9, 1, 2, 6, 9, 1, 9, 9, 9, 1, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 2, 1, 9, 7, 9, 2, 1, 9, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 9, 6, 2, 2,
       0, 2, 2, 2, 2, 2, 2, 9, 2, 2, 2, 4, 2, 9, 2, 2, 1, 2, 2, 1, 2, 3,
       0, 1, 3, 3, 2, 2, 3, 3, 3, 2, 1, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       0, 4, 4, 4, 1, 9, 2, 6, 9, 6, 1])

In [37]:
hist = model_new.fit(X_train, 
                 Y_train,
                 shuffle = True,
                 batch_size=50,
                 epochs =2,
                 steps_per_epoch=30,
                 validation_split = 0.20)

Epoch 1/2


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [50,10] and labels shape [500]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-37-ecfb5202df1e>:7) ]] [Op:__inference_train_function_1814]

Function call stack:
train_function


In [363]:
folders = os.listdir("Test/images")
print(folders)

['test_0.jpg', 'test_1.jpg', 'test_10.jpg', 'test_100.jpg', 'test_101.jpg', 'test_102.jpg', 'test_103.jpg', 'test_104.jpg', 'test_105.jpg', 'test_106.jpg', 'test_107.jpg', 'test_108.jpg', 'test_109.jpg', 'test_11.jpg', 'test_110.jpg', 'test_111.jpg', 'test_112.jpg', 'test_113.jpg', 'test_114.jpg', 'test_115.jpg', 'test_116.jpg', 'test_117.jpg', 'test_118.jpg', 'test_119.jpg', 'test_12.jpg', 'test_120.jpg', 'test_121.jpg', 'test_122.jpg', 'test_123.jpg', 'test_124.jpg', 'test_125.jpg', 'test_126.jpg', 'test_127.jpg', 'test_128.jpg', 'test_129.jpg', 'test_13.jpg', 'test_130.jpg', 'test_131.jpg', 'test_132.jpg', 'test_133.jpg', 'test_134.jpg', 'test_135.jpg', 'test_136.jpg', 'test_137.jpg', 'test_138.jpg', 'test_139.jpg', 'test_14.jpg', 'test_140.jpg', 'test_141.jpg', 'test_142.jpg', 'test_143.jpg', 'test_144.jpg', 'test_145.jpg', 'test_146.jpg', 'test_147.jpg', 'test_148.jpg', 'test_149.jpg', 'test_15.jpg', 'test_150.jpg', 'test_151.jpg', 'test_152.jpg', 'test_153.jpg', 'test_154.jpg', '

In [364]:
from pathlib import Path

In [365]:
py = Path("Test/images")

In [453]:
image_datay = []
imagenames = []
for img_path in py.glob("*.jpg"):
    label = str(img_path).split("\\")[-1]
    print(label)
    imagenames.append(label)
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    image_datay.append(img_array)


test_0.jpg
test_1.jpg
test_10.jpg
test_100.jpg
test_101.jpg
test_102.jpg
test_103.jpg
test_104.jpg
test_105.jpg
test_106.jpg
test_107.jpg
test_108.jpg
test_109.jpg
test_11.jpg
test_110.jpg
test_111.jpg
test_112.jpg
test_113.jpg
test_114.jpg
test_115.jpg
test_116.jpg
test_117.jpg
test_118.jpg
test_119.jpg
test_12.jpg
test_120.jpg
test_121.jpg
test_122.jpg
test_123.jpg
test_124.jpg
test_125.jpg
test_126.jpg
test_127.jpg
test_128.jpg
test_129.jpg
test_13.jpg
test_130.jpg
test_131.jpg
test_132.jpg
test_133.jpg
test_134.jpg
test_135.jpg
test_136.jpg
test_137.jpg
test_138.jpg
test_139.jpg
test_14.jpg
test_140.jpg
test_141.jpg
test_142.jpg
test_143.jpg
test_144.jpg
test_145.jpg
test_146.jpg
test_147.jpg
test_148.jpg
test_149.jpg
test_15.jpg
test_150.jpg
test_151.jpg
test_152.jpg
test_153.jpg
test_154.jpg
test_155.jpg
test_156.jpg
test_157.jpg
test_158.jpg
test_159.jpg
test_16.jpg
test_160.jpg
test_161.jpg
test_162.jpg
test_163.jpg
test_164.jpg
test_165.jpg
test_166.jpg
test_167.jpg
test_168.j

In [454]:
X_test = np.array(image_datay)

In [456]:
predictions = model_new.predict(X_test)

In [457]:
type(predictions)

numpy.ndarray

In [461]:
predictions[:10]

array([[1.03799696e-03, 1.62578702e-01, 2.29534358e-01, 1.56259835e-02,
        1.84799777e-03, 1.91970867e-05, 5.31319916e-01, 5.42507619e-02,
        2.75206985e-04, 3.50992172e-03],
       [1.96422679e-05, 1.42249063e-01, 8.45597148e-01, 2.07891819e-04,
        3.12149408e-04, 5.80379947e-07, 1.07316766e-02, 1.20215904e-04,
        6.39743894e-06, 7.55205983e-04],
       [2.19978785e-04, 1.33305986e-03, 1.16040027e-02, 1.04188462e-06,
        2.60446313e-07, 1.50661106e-06, 9.86825287e-01, 1.28195625e-05,
        7.46054862e-09, 1.97184318e-06],
       [5.18192954e-12, 2.20265074e-11, 1.70345942e-08, 3.08033894e-12,
        2.06306196e-12, 8.67719592e-14, 1.00000000e+00, 6.59378761e-14,
        6.00698190e-16, 1.75884741e-12],
       [3.45372155e-06, 9.17249262e-01, 8.99219885e-03, 1.99538863e-06,
        2.34635223e-07, 3.59822527e-09, 2.25580006e-05, 7.37144649e-02,
        3.97633482e-10, 1.59046431e-05],
       [1.01702993e-10, 5.23449620e-03, 9.89072084e-01, 3.66849775e-07,
   

In [299]:
imagenames

['test_0.jpg',
 'test_1.jpg',
 'test_10.jpg',
 'test_100.jpg',
 'test_101.jpg',
 'test_102.jpg',
 'test_103.jpg',
 'test_104.jpg',
 'test_105.jpg',
 'test_106.jpg',
 'test_107.jpg',
 'test_108.jpg',
 'test_109.jpg',
 'test_11.jpg',
 'test_110.jpg',
 'test_111.jpg',
 'test_112.jpg',
 'test_113.jpg',
 'test_114.jpg',
 'test_115.jpg',
 'test_116.jpg',
 'test_117.jpg',
 'test_118.jpg',
 'test_119.jpg',
 'test_12.jpg',
 'test_120.jpg',
 'test_121.jpg',
 'test_122.jpg',
 'test_123.jpg',
 'test_124.jpg',
 'test_125.jpg',
 'test_126.jpg',
 'test_127.jpg',
 'test_128.jpg',
 'test_129.jpg',
 'test_13.jpg',
 'test_130.jpg',
 'test_131.jpg',
 'test_132.jpg',
 'test_133.jpg',
 'test_134.jpg',
 'test_135.jpg',
 'test_136.jpg',
 'test_137.jpg',
 'test_138.jpg',
 'test_139.jpg',
 'test_14.jpg',
 'test_140.jpg',
 'test_141.jpg',
 'test_142.jpg',
 'test_143.jpg',
 'test_144.jpg',
 'test_145.jpg',
 'test_146.jpg',
 'test_147.jpg',
 'test_148.jpg',
 'test_149.jpg',
 'test_15.jpg',
 'test_150.jpg',
 'test_

In [468]:
Y_test = np.argmax(predictions, axis=1)

In [552]:
Y_test = predictions

In [553]:
Y_test[:10]

array([0, 0, 1, 6, 6, 6, 6, 6, 6, 9])

In [489]:
Y_test.shape

(187,)

In [554]:
a = np.array(imagenames)
b = Y_test
a = a.reshape(a.shape[0],-1)
a.shape
b = b.reshape(b.shape[0],-1)

In [555]:
result = np.zeros((Y_test.shape[0],2))

In [556]:
result = np.hstack((a,b))

In [557]:
import pandas as pd

In [558]:
df = pd.DataFrame(data=result, columns=["Name","Class"])
df.head()
df.to_csv('test.csv')

In [559]:
df.head()

Name Class
0    test_0.jpg     0
1    test_1.jpg     0
2   test_10.jpg     1
3  test_100.jpg     6
4  test_101.jpg     6

In [560]:
YTest = pd.read_csv("Test/sample_submission.csv")
YTest.head()
YTest.drop(columns=['Class'])

Name
0     test_32.jpg
1    test_147.jpg
2    test_171.jpg
3    test_114.jpg
4    test_162.jpg
..            ...
182  test_150.jpg
183   test_72.jpg
184   test_95.jpg
185   test_88.jpg
186  test_110.jpg

[187 rows x 1 columns]

In [561]:
Yd = pd.merge(df, YTest, how='right', left_on='Name', right_on='Name')

In [562]:
Yd.head()

Name Class_x  Class_y
0   test_32.jpg       1        0
1  test_147.jpg       8        0
2  test_171.jpg       9        0
3  test_114.jpg       6        0
4  test_162.jpg       1        0

In [563]:
Yd.to_csv('ans.csv')